In [2]:
!nvidia-smi

Mon Jun 12 23:38:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.81                 Driver Version: 460.67                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:85:00.0 Off |                  N/A |
| 23%   32C    P8    10W / 250W |      2MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

tf.get_logger().setLevel('ERROR')

2023-06-12 23:38:22.516658: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 23:38:22.697456: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-12 23:38:22.697506: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-12 23:38:24.422616: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7:

In [4]:
# df = pd.read_csv("../data/cases_f_cleaned.csv")
# df = df.loc[df["u_organization"] == "IT Services", ["description", "service_offering"]]
# top_25_services = df.groupby("service_offering").count().sort_values(by="description", ascending=False)[:25].index
# df["service_offering"] = df["service_offering"].apply(lambda x: x if x in top_25_services else "Other")
# df["description"] = df["description"].fillna("")
# df.to_csv("../data/processed_its_top_25.csv")

In [5]:
filename = "../data/processed_its_top_25.csv"

In [6]:
df = pd.read_csv(filename)
df["description"] = df["description"].fillna("")
X = df["description"].fillna("None").to_numpy()
le = LabelEncoder()
y = le.fit(df["service_offering"])

In [7]:
# Sample 800 descriptions from each of the 25 service_offering and the other categories
groups = df.groupby("service_offering")["description"]
dataset = {"X":[], "y":[]}
for service, group in groups:
    dataset["y"].extend([service for _ in range(800)])
    dataset["X"].extend(group.sample(800).values)
X_train, X_val, y_train, y_val = train_test_split(np.array(dataset["X"]), dataset["y"], test_size = 0.2, random_state=42, stratify=dataset["y"])

In [8]:
len(X_train), len(X_val), len(y_train), len(y_val)

(16640, 4160, 16640, 4160)

In [9]:
le.inverse_transform(y_val)

ValueError: y contains previously unseen labels: ['Access to Business Applications' 'Active Directory - ITS'
 'Campus Wired Networking' 'Campus Wireless Networking' 'Canvas LMS'
 'Domain Name Registration' 'Enterprise Software Licensing Support'
 'Exchange Online' 'Hardware Endpoint Support' 'Health IT Services '
 'IP Address Management (hostmaint, ethernet registration form) '
 'MultiFactor Authentication' 'Non ITS Support' 'Office 365'
 'Online Directory' 'Other' 'Password reset'
 'Platform Operations & Management ' 'SSL / Digital Certificates'
 'Single Sign-on' 'Software Endpoint Support' 'Spam / Duplicate Case'
 'Virtual Private Network (VPN)' 'Web & Video Conferencing (Zoom)'
 'Web Content Management' 'Wireless Residential Networking']

In [ ]:
val_df = pd.DataFrame(data={"desc": X_val, "service_offering":le.inverse_transform(y_val)})

In [ ]:
val_df.head()

In [ ]:
le.inverse_transform([0,1])

In [ ]:
np.save("services.npy", le.classes_)

In [ ]:
val_df.to_csv("../data/validation_set.csv")

In [ ]:
print(np.unique(np.array(y_val), return_counts=True))

In [ ]:
print(len(X_train),len(X_val), len(y_train), len(y_val))
# Validate that the split was done successfully
print(np.unique(np.array(y_train), return_counts=True))
# Downside of this approach is that it massivley undersamples Other categories
y_train, y_val = le.transform(y_train), le.transform(y_val)
num_classes = len(le.classes_)

In [ ]:
# df = pd.read_csv("../data/processed_its_top_25.csv")
# df["description"] = df["description"].fillna("")
# X = df["description"].fillna("None").to_numpy()
# le = LabelEncoder()
# y = le.fit_transform(df["service_offering"])
# num_classes = len(le.classes_)
# # Use the following line if you plan to use CategoricalCrossentropy as the loss function
# # y = tf.keras.utils.to_categorical(y, num_classes=num_classes)
# X_discard, X, y_discard, y = train_test_split(X, y, test_size=0.05, random_state=42, stratify=y)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-256_A-4' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name="preprocessing")
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name="BERT_encoder")
    outputs = encoder(encoder_inputs)
    net = outputs["pooled_output"]
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(num_classes, activation="softmax", name="classifier")(net)
    return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
bert_raw_results = classifier_model(X_train)

In [ ]:
classifier_model.summary()

In [ ]:
def f1_score(y_true, y_pred):
    y_true = tf.cast(y_true, "float32")
    y_pred = tf.argmax(y_pred, axis=-1)
    tp = tf.reduce_sum(tf.cast(y_true*y_pred, "float32"), axis=0)
    fp = tf.reduce_sum(tf.cast((1-y_true)*y_pred, "float32"), axis=0)
    fn = tf.reduce_sum(tf.cast(y_true*(1-y_pred), "float32"), axis=0)
    precision = tp(tp+fp)
    recall = tp/(tp+fn)
    f1 = 2 * (precision*recall)/(precision+recall)
    return tf.reduce_mean(f1)

In [ ]:
# # Categorical Cross Entropy and Categorical Accuracy
# loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
# metrics = tf.keras.metrics.CategoricalAccuracy()

In [ ]:
# Sparse Categorical Cross Entropy and F1 Score
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
metrics = tf.keras.metrics.CategoricalAccuracy()

In [ ]:
epochs = 30
steps_per_epoch = 1000
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type="adamw"
)

In [ ]:
classifier_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["accuracy"]
)

In [ ]:
# batch_sizes = [8, 16, 32, 64, 128]
# learning_rates: [3e-4, 1e-4, 5e-5, 3e-5]
# sample_weights

In [ ]:
# Callback for early termination
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode="max",
    verbose=1,
    patience=3
)

In [ ]:
checkpoint_path = "model_checkpoint/BERT_25_{epoch:02d}_{val_accuracy:.2f}.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
classifier_model.load_weights("model_checkpoint/BERT_25_01_1.30.hdf5")

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    callbacks=[es, model_checkpoint_callback]
)

In [ ]:
classifier_model.save_weights('my_model_weights.h5')

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [ ]:
def predict(mdl, desc, le):
    probabilities = np.argmax(mdl.predict(x = test))
    return le.inverse_transform([pred])

In [ ]:
predict(classifier_model, test, le)

In [ ]:
test = ["My Duo Mobile app is not working. It says that the account is disabled. Please contact the Service Desk."]
pred = np.argmax(classifier_model.predict(x = test))
le.inverse_transform([pred])

In [ ]:
test = ["My account is compromised, I need to change my password"]
pred = np.argmax(classifier_model.predict(x = test))
le.inverse_transform([pred])

In [ ]:
test = ["Wifi at Sixth college is not working. All of my friends are reporting a problem"]
pred = np.argmax(classifier_model.predict(x = test))
le.inverse_transform([pred])

In [ ]:
test = ["My VPN is not working. It is stuck on trying to connect when I type in my password"]
pred = np.argmax(classifier_model.predict(x = test))
le.inverse_transform([pred])

In [ ]:
test = ["I am not receiving any emails in my Outlook exchange. The last email I got was 2 months ago. HELP!!!"]
pred = np.argmax(classifier_model.predict(x = test))
le.inverse_transform([pred])

In [ ]:
def predict_class(probs):
    return [np.argmax(pred) for pred in model.predict(probs)]

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
text_preprocessed = bert_preprocess_model(X_small)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

In [ ]:
tf.keras.utils.plot_model(classifier_model)